In [14]:
#!jupyter nbconvert --to script down_pdf.ipynb
import requests
#url = 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2010-03-31/600795_2009_n.pdf'
url = 'http://disc.static.szse.cn/download/disc/disk01/finalpage/2008-04-19/cda5e777-cc3c-4934-9971-99ce9e4f6fe0.PDF'
def down_pdf(filename,url):
    success = True
    try:
        r = requests.get(url, stream=True)
        with open(filename, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=102400):
                fd.write(chunk)
    except Exception as e:
        success = False
        print(f'download {url} faild',e)
        pass
    
    return filename,success




In [15]:
# 检查文件合法性
import os
from PyPDF2 import PdfFileWriter, PdfFileReader
def checkPdf(f):
    """检查pdf文件的合法性"""
    # 文件是否存在
    if not os.path.exists(f):
        return False
    
    # 文件大小是否满足条件
    fsize=0
    try:
        fsize = os.path.getsize(f)
    except Exception as e:
        print(f'getsize exception {f} ',e)
        return False

    if fsize < 1024*50:
        return False
    
    # 是否正常的pdf文件
    try:
        pdfFileReader = PdfFileReader(f)
        if pdfFileReader.isEncrypted:
            pdfFileReader.decrypt('')
        if pdfFileReader.getDocumentInfo()  is not None:
            return True
    except:
        pass
    return False
    

In [19]:
import time
import json
from tqdm import tqdm

def down_loop():
    condition = int(os.environ.get("myCondition",'0'))
    print(condition)
    with open('pdf_urls.json') as json_file:  
        pdf_urls = json.load(json_file)

    keys = list(pdf_urls.keys())
    base_dir='../scraping/content/'
    pdf_dir ='../scraping/pdf/'
    for fileName in tqdm(keys):
        code = int(fileName[:6])
        url = pdf_urls[fileName]
        if code % 4 !=condition:
#             print('jump',code)
            continue

        pdf_file=pdf_dir+fileName+'.pdf'
        if checkPdf(pdf_file):
            print(f'{pdf_file} is ok,jump...')
            continue

        times = 0
        while times<= 3:
            times +=1
            t0=time.time()
            print(f'开始下载文件：{url},times={times}')
            pdf_file,success = down_pdf(pdf_file,url)
            if not success:
                time.sleep(2)
                continue
            fsize=1
            try:
                fsize = os.path.getsize(pdf_file)
            except:
                pass
            print(f'下载文件({pdf_file})成功，用时(秒):{time.time()-t0},times={times},fsize={fsize/1024}KB')
            t0=time.time()
            if checkPdf(pdf_file):
                print('+'*30,f'校验文件({pdf_file})成功')
                break
            else:
                print('-'*30,f'校验文件({pdf_file})失败，times={times}')
    
    time.sleep(120)
    
    
down_loop()
    


  1%|          | 43/4660 [00:00<00:11, 395.36it/s]

0
jump 1
jump 1
jump 1
jump 1
jump 1
jump 1
jump 1
jump 1
jump 1
jump 1
jump 1
jump 1
jump 1
jump 1
jump 1
jump 1
jump 1
jump 1
jump 2
jump 2
jump 2
jump 2
jump 2
jump 2
jump 2
jump 2
jump 2
jump 2
jump 2
jump 2
jump 2
jump 2
jump 2
jump 2
jump 2
../scraping/pdf/000004_2001.pdf is ok,jump...
../scraping/pdf/000004_2002.pdf is ok,jump...
../scraping/pdf/000004_2003.pdf is ok,jump...
../scraping/pdf/000004_2004.pdf is ok,jump...
../scraping/pdf/000004_2005.pdf is ok,jump...
../scraping/pdf/000004_2006.pdf is ok,jump...
../scraping/pdf/000004_2007.pdf is ok,jump...
../scraping/pdf/000004_2011.pdf is ok,jump...
../scraping/pdf/000004_2012.pdf is ok,jump...
../scraping/pdf/000004_2013.pdf is ok,jump...
../scraping/pdf/000004_2015.pdf is ok,jump...



  2%|▏         | 111/4660 [00:00<00:21, 211.16it/s]

../scraping/pdf/000004_2017.pdf is ok,jump...
../scraping/pdf/000004_2018.pdf is ok,jump...
jump 5
jump 5
jump 5
jump 5
jump 5
jump 5
jump 5
jump 5
jump 5
jump 5
jump 5
jump 5
jump 5
jump 5
jump 5
jump 5
jump 6
jump 6
jump 6
jump 6
jump 6
jump 6
jump 6
jump 6
jump 6
jump 6
jump 6
jump 6
jump 6
jump 6
jump 6
jump 6
jump 7
jump 7
jump 7
jump 7
jump 7
jump 7
jump 7
jump 7
jump 7
jump 7
jump 7
jump 7
jump 7
jump 7
jump 7
../scraping/pdf/000008_2001.pdf is ok,jump...
../scraping/pdf/000008_2002.pdf is ok,jump...
../scraping/pdf/000008_2003.pdf is ok,jump...
../scraping/pdf/000008_2004.pdf is ok,jump...
../scraping/pdf/000008_2005.pdf is ok,jump...
../scraping/pdf/000008_2006.pdf is ok,jump...
../scraping/pdf/000008_2007.pdf is ok,jump...
../scraping/pdf/000008_2008.pdf is ok,jump...
../scraping/pdf/000008_2009.pdf is ok,jump...
../scraping/pdf/000008_2010.pdf is ok,jump...
../scraping/pdf/000008_2011.pdf is ok,jump...
../scraping/pdf/000008_2012.pdf is ok,jump...
../scraping/pdf/000008_2015


  3%|▎         | 161/4660 [00:02<01:10, 63.50it/s] 

...


  8%|▊         | 383/4660 [00:22<27:06,  2.63it/s]

下载文件(../scraping/pdf/000028_2014.pdf)成功，用时(秒):2.0317935943603516,times=1,fsize=712.0KB
++++++++++++++++++++++++++++++ 校验文件(../scraping/pdf/000028_2014.pdf)成功
开始下载文件：http://disc.static.szse.cn/download/disc/disk01/finalpage/2008-04-19/cda5e777-cc3c-4934-9971-99ce9e4f6fe0.PDF,times=1
下载文件(../scraping/pdf/000028_2015.pdf)成功，用时(秒):0.035573720932006836,times=1,fsize=712.0KB
++++++++++++++++++++++++++++++ 校验文件(../scraping/pdf/000028_2015.pdf)成功
开始下载文件：http://disc.static.szse.cn/download/disc/disk01/finalpage/2008-04-19/cda5e777-cc3c-4934-9971-99ce9e4f6fe0.PDF,times=1
下载文件(../scraping/pdf/000028_2016.pdf)成功，用时(秒):0.030759096145629883,times=1,fsize=712.0KB
++++++++++++++++++++++++++++++ 校验文件(../scraping/pdf/000028_2016.pdf)成功
开始下载文件：http://disc.static.szse.cn/download/disc/disk01/finalpage/2008-04-19/cda5e777-cc3c-4934-9971-99ce9e4f6fe0.PDF,times=1



  8%|▊         | 387/4660 [00:24<30:02,  2.37it/s]

下载文件(../scraping/pdf/000028_2017.pdf)成功，用时(秒):2.0308523178100586,times=1,fsize=712.0KB
++++++++++++++++++++++++++++++ 校验文件(../scraping/pdf/000028_2017.pdf)成功
开始下载文件：http://disc.static.szse.cn/download/disc/disk01/finalpage/2008-04-19/cda5e777-cc3c-4934-9971-99ce9e4f6fe0.PDF,times=1
下载文件(../scraping/pdf/000028_2018.pdf)成功，用时(秒):0.03535294532775879,times=1,fsize=712.0KB
++++++++++++++++++++++++++++++ 校验文件(../scraping/pdf/000028_2018.pdf)成功
jump 29
jump 29
jump 29
jump 29
jump 29
jump 29
jump 29
jump 29
jump 29
jump 29
jump 29
jump 29
jump 29
jump 29
jump 29
jump 29
jump 29
jump 30
jump 30
jump 30
jump 30
jump 30
jump 30
jump 30
jump 30
jump 30
jump 30
jump 30
jump 30
jump 30
jump 30
jump 30
jump 30
jump 30
jump 30
jump 31
jump 31
jump 31
jump 31
jump 31
jump 31
jump 31
jump 31
jump 31
jump 31
jump 31
jump 31
jump 31
jump 31
jump 31
jump 31
jump 31
jump 31
开始下载文件：http://disc.static.szse.cn/download/disc/disk01/finalpage/2008-04-19/cda5e777-cc3c-4934-9971-99ce9e4f6fe0.PDF,times=1
下载文件(../s

KeyboardInterrupt: 